In [1]:
import pandas as pd

# 1. Load the data
# specify your file path
path = "metadata_master.xlsx"
df = pd.read_excel(path, sheet_name=0, dtype="object")

print("Original table:", df.shape)

# 2. Keep only important columns
keep_cols = [
    # identification
    "dataset", "sample_id", "participant_id", "condition",
    "unit_level", "modality",

    # features/targets
    "movement_intensity_raw", "movement_intensity_z", "movement_intensity_bin",
    "engagement_level",

    # fairness / covariates
    "sex", "age_years", "age_group",

    # questionnaires
    "intervention_type", "elapsed_time_sec_total",
    "sus_total", "nasa_tlx_weighted", "nasa_tlx_unweighted",

    # splits
    "split_seed", "split_iid", "split_lodo",

    # blocks (keep for potential analysis)
    "block_field", "block_id",]

# remove those missing in the specific table
keep_cols = [c for c in keep_cols if c in df.columns]

df_ml = df[keep_cols].copy()

print("After cleaning:", df_ml.shape)

# 3. Show first rows and info
display(df_ml.head(10))
print("\nColumn information:")
print(df_ml.info())

# 4. Save ML-ready table
df_ml.to_excel("metadata_ml_ready.xlsx", index=False, sheet_name="ml_ready")
df_ml.to_csv("metadata_ml_ready.csv", index=False, encoding="utf-8-sig")
print("\nFiles saved: metadata_ml_ready.xlsx / metadata_ml_ready.csv")

Исходная таблица: (2131, 35)
После очистки: (2131, 18)


,dataset,sample_id,participant_id,condition,unit_level,modality,movement_intensity_raw,movement_intensity_z,movement_intensity_bin,engagement_level,sex,age_years,age_group,split_seed,split_iid,split_lodo,block_field,block_id
0,MMASD,as_20583_D16_000_i,20583,NaN,NaN,NaN,0.997162,0.340884,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
1,MMASD,as_20583_D16_001_n,20583,NaN,NaN,NaN,0.962013,0.000105,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
2,MMASD,as_20583_D16_002_n,20583,NaN,NaN,NaN,0.953138,-0.085941,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
3,MMASD,as_20583_D1_000_y,20583,NaN,NaN,NaN,1.018049,0.543383,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
4,MMASD,as_20583_D1_001_y,20583,NaN,NaN,NaN,1.098037,1.318874,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
5,MMASD,as_20583_D1_002_i,20583,NaN,NaN,NaN,1.050288,0.855942,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
6,MMASD,as_20583_D1_003_i,20583,NaN,NaN,NaN,0.047281,-8.86833,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
7,MMASD,as_20583_D8_000_i,20583,NaN,NaN,NaN,0.980377,0.178151,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
8,MMASD,as_20583_D8_001_i,20583,NaN,NaN,NaN,0.943045,-0.183791,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
9,MMASD,as_20583_D8_002_n,20583,NaN,NaN,NaN,1.013126,0.495656,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN



Информация по колонкам:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2131 entries, 0 to 2130
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   dataset                 2131 non-null   object
 1   sample_id               2131 non-null   object
 2   participant_id          2131 non-null   object
 3   condition               438 non-null    object
 4   unit_level              438 non-null    object
 5   modality                438 non-null    object
 6   movement_intensity_raw  1431 non-null   object
 7   movement_intensity_z    1431 non-null   object
 8   movement_intensity_bin  1750 non-null   object
 9   engagement_level        0 non-null      object
 10  sex                     1268 non-null   object
 11  age_years               1268 non-null   object
 12  age_group               0 non-null      object
 13  split_seed              0 non-null      object
 14  split_iid               0 non-n

In [1]:
import pandas as pd
import numpy as np

# 1. Load
PATH_IN = "metadata_ml_ready.xlsx"
SHEET   = 0

if PATH_IN.lower().endswith(".csv"):
    df = pd.read_csv(PATH_IN, dtype="object")
else:
    df = pd.read_excel(PATH_IN, sheet_name=SHEET, dtype="object")

print("Original:", df.shape)

# 2. Normalize and convert to numeric for the two key metrics
KEYS = ["movement_intensity_raw", "movement_intensity_z"]
for c in KEYS:
    if c not in df.columns:
        df[c] = pd.NA
    df[c] = df[c].replace(r"^\s*$", np.nan, regex=True) \
                 .replace({"NA": np.nan, "NaN": np.nan, "nan": np.nan,
                           "None": np.nan, "null": np.nan})
    df[c] = pd.to_numeric(df[c], errors="coerce")

# 3. Keep rows where BOTH metrics are present
mask = df["movement_intensity_raw"].notna() & df["movement_intensity_z"].notna()
removed = int((~mask).sum())
df_clean = df.loc[mask].copy()

print(f"Removed rows (missing in {KEYS}): {removed}")
print("Cleaned:", df_clean.shape)

# 4. Quick checks
if "dataset" in df_clean.columns:
    print("\nRows by dataset:")
    print(df_clean["dataset"].value_counts(dropna=False))

if "movement_intensity_bin" in df_clean.columns:
    print("\nClass balance (movement_intensity_bin):")
    print(pd.to_numeric(df_clean["movement_intensity_bin"], errors="coerce")
            .value_counts(dropna=False))

display(df_clean.head(10))

# 5. Save
OUT_XLSX = "metadata_ml_ready_clean_2metrics.xlsx"
OUT_CSV  = "metadata_ml_ready_clean_2metrics.csv"
df_clean.to_excel(OUT_XLSX, index=False, sheet_name="ml_ready_clean")
df_clean.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print(f"\nSaved: {OUT_XLSX}  /  {OUT_CSV}")


Original: (2131, 18)
Removed rows (missing in ['movement_intensity_raw', 'movement_intensity_z']): 700
Cleaned: (1431, 18)

Rows by dataset:
dataset
MMASD          1374
Engagnition      57
Name: count, dtype: int64

Class balance (movement_intensity_bin):
movement_intensity_bin
1    724
0    707
Name: count, dtype: int64


C:\Users\rusla\AppData\Local\Temp\ipykernel_22388\3700100355.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[c] = df[c].replace(r"^\s*$", np.nan, regex=True) \


,dataset,sample_id,participant_id,condition,unit_level,modality,movement_intensity_raw,movement_intensity_z,movement_intensity_bin,engagement_level,sex,age_years,age_group,split_seed,split_iid,split_lodo,block_field,block_id
0,MMASD,as_20583_D16_000_i,20583,NaN,NaN,NaN,0.997162,0.340884,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
1,MMASD,as_20583_D16_001_n,20583,NaN,NaN,NaN,0.962013,0.000105,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
2,MMASD,as_20583_D16_002_n,20583,NaN,NaN,NaN,0.953138,-0.085941,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
3,MMASD,as_20583_D1_000_y,20583,NaN,NaN,NaN,1.018049,0.543383,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
4,MMASD,as_20583_D1_001_y,20583,NaN,NaN,NaN,1.098037,1.318874,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
5,MMASD,as_20583_D1_002_i,20583,NaN,NaN,NaN,1.050288,0.855942,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
6,MMASD,as_20583_D1_003_i,20583,NaN,NaN,NaN,0.047281,-8.868330,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
7,MMASD,as_20583_D8_000_i,20583,NaN,NaN,NaN,0.980377,0.178151,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
8,MMASD,as_20583_D8_001_i,20583,NaN,NaN,NaN,0.943045,-0.183791,0,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN
9,MMASD,as_20583_D8_002_n,20583,NaN,NaN,NaN,1.013126,0.495656,1,NaN,M,"5y, 9m",NaN,NaN,NaN,NaN,NaN,NaN



Saved: metadata_ml_ready_clean_2metrics.xlsx  /  metadata_ml_ready_clean_2metrics.csv


In [2]:
import pandas as pd
import numpy as np

# 0. Load the cleaned table
PATH_IN = "metadata_ml_ready_clean_2metrics.xlsx"
SHEET   = 0

if PATH_IN.lower().endswith(".csv"):
    df = pd.read_csv(PATH_IN, dtype="object")
else:
    df = pd.read_excel(PATH_IN, sheet_name=SHEET, dtype="object")

print("Input shape:", df.shape)

# 1. Final filtering for training
# keep only rows with a valid target
df["movement_intensity_bin"] = pd.to_numeric(df["movement_intensity_bin"], errors="coerce")
df = df[df["movement_intensity_bin"].notna()].copy()

# normalize participant_id and condition (Baseline/LPE/HPE/NA)
def norm_pid(x):
    s = str(x).upper().strip()
    m = pd.Series([s]).str.extract(r"(P\d+)", expand=False).iloc[0]
    return m if pd.notna(m) else s

def norm_cond(x):
    s = str(x).strip().lower()
    if s in ("hpe", "highphysicalengagement", "high-physical-engagement"): return "HPE"
    if s in ("lpe", "lowphysicalengagement", "low-physical-engagement"):   return "LPE"
    if "base" in s: return "Baseline"
    return "NA"

if "participant_id" in df.columns:
    df["participant_id"] = df["participant_id"].apply(norm_pid)

if "condition" in df.columns:
    df["condition"] = df["condition"].apply(norm_cond)

# remove empty participant_id
df = df[df["participant_id"].notna() & (df["participant_id"] != "")].copy()

# optionally remove duplicate sample_id
if "sample_id" in df.columns:
    dup = df["sample_id"].duplicated(keep="first")
    if dup.any():
        print(f"Removed duplicate sample_id rows: {int(dup.sum())}")
        df = df[~dup].copy()

print("After target & ID cleanup:", df.shape)

# 2. Prepare stratification label and groups, stratify by dataset × class, group by participant_id
df["dataset"] = df.get("dataset", "NA").astype(str)
y  = df["movement_intensity_bin"].astype(int).values
grp = df["participant_id"].astype(str).values
strata = (df["dataset"] + "_" + df["movement_intensity_bin"].astype(int).astype(str)).values

# 3. Generate k-folds with grouped stratification
# Needs sklearn>=1.1 for StratifiedGroupKFold, fallback = GroupKFold
try:
    from sklearn.model_selection import StratifiedGroupKFold
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
    fold_idx = np.full(len(df), -1, dtype=int)
    for k, (_, val_idx) in enumerate(sgkf.split(np.zeros(len(df)), y=strata, groups=grp)):
        fold_idx[val_idx] = k
    df["cv_fold_iid"] = fold_idx
except Exception as e:
    print("[WARN] StratifiedGroupKFold not available, falling back to GroupKFold.", e)
    from sklearn.model_selection import GroupKFold
    gkf = GroupKFold(n_splits=5)
    fold_idx = np.full(len(df), -1, dtype=int)
    for k, (_, val_idx) in enumerate(gkf.split(np.zeros(len(df)), y, groups=grp)):
        fold_idx[val_idx] = k
    df["cv_fold_iid"] = fold_idx

# map folds to train/val/test: 0→test, 1→val, others→train
df["split_iid"] = np.where(df["cv_fold_iid"] == 0, "test",
                    np.where(df["cv_fold_iid"] == 1, "val", "train"))

# LODO split (GroupKFold-like leave-one-domain-out)
# map participant_id to a stable hash-based fold
pid_codes = pd.factorize(df["participant_id"])[0]
df["cv_fold_lodo"] = (pd.util.hash_pandas_object(df["participant_id"], index=False) % 5).astype(int)
df["split_lodo"] = np.where(df["cv_fold_lodo"] == 0, "test",
                     np.where(df["cv_fold_lodo"] == 1, "val", "train"))

# optional: fix random seed
df["split_seed"] = 42

# 4. Diagnostics of splits
def summary(name):
    tab = (df.groupby([name, "dataset", "movement_intensity_bin"])
             .size()
             .unstack(fill_value=0)
             .rename_axis(columns="bin"))
    print(f"\n{name} distribution (dataset × bin):")
    print(tab)

summary("split_iid")
summary("split_lodo")

print("\nRows per split (iid):")
print(df["split_iid"].value_counts())

# 5. Save
df.to_excel("metadata_ml_ready_splits.xlsx", index=False, sheet_name="ml_ready_splits")
df.to_csv("metadata_ml_ready_splits.csv", index=False, encoding="utf-8-sig")
print("\nSaved: metadata_ml_ready_splits.xlsx / metadata_ml_ready_splits.csv")


Input shape: (1431, 18)
After target & id cleanup: (1431, 18)

split_iid distribution (dataset × bin):
bin                      0    1
split_iid dataset              
test      Engagnition    7    6
          MMASD         70   72
train     Engagnition   12   20
          MMASD        494  504
val       Engagnition    9    3
          MMASD        115  119

split_lodo distribution (dataset × bin):
bin                       0    1
split_lodo dataset              
test       Engagnition    7    5
           MMASD        145  150
train      Engagnition   13   16
           MMASD        373  381
val        Engagnition    8    8
           MMASD        161  164

Rows per split (iid):
split_iid
train    1030
val       246
test      155
Name: count, dtype: int64

Saved: metadata_ml_ready_splits.xlsx / metadata_ml_ready_splits.csv


In [3]:
# Check: one participant_id should not appear in both train and test/val
for split in ["split_iid", "split_lodo"]:
    for a, b in [("train", "test"), ("train", "val"), ("val", "test")]:
        A = set(df.loc[df[split] == a, "participant_id"])
        B = set(df.loc[df[split] == b, "participant_id"])
        inter = A & B
        print(f"{split}: {a} ∩ {b} = {len(inter)}")


split_iid: train∩test = 0
split_iid: train∩val = 0
split_iid: val∩test = 0
split_lodo: train∩test = 0
split_lodo: train∩val = 0
split_lodo: val∩test = 0


In [4]:
import json

split_info = {}
for split in ["split_iid", "split_lodo"]:
    split_info[split] = {}
    for part in ["train", "val", "test"]:
        split_info[split][part] = sorted(
            df.loc[df[split] == part, "participant_id"].unique().tolist())

with open("splits_manifest.json", "w", encoding="utf-8") as f:
    json.dump(split_info, f, ensure_ascii=False, indent=2)

print("Saved splits_manifest.json")

Saved splits_manifest.json
